# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import os

# Import API key
from api_keys import geoapify_key

In [2]:
#load csv file
part1file = 'output_data/cities.csv'  

# Read csv cities file and store into data frame:
citydata = pd.read_csv(part1file)

vacationpydf = pd.DataFrame(citydata)
vacationpydf.head()

,City_ID,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,Waitangi,NZ,1733443034,-43.9535,-176.5597,78,82,57.81,13.00
1,1,Bethel,US,1733443035,41.3712,-73.4140,49,63,35.01,5.01
2,2,Cabo San Lucas,MX,1733442969,22.8909,-109.9124,40,55,74.77,8.39
3,3,Olonkinbyen,SJ,1733443036,70.9221,-8.7187,100,99,28.74,21.92
4,4,Alotau,PG,1733443036,-10.3069,150.4462,100,67,86.95,3.58


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = citydata.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = citydata[citydata['Max Temp (F)'] >=21]

# Drop any rows with null values
new_city_df.dropna()

# Display sample data
new_city_df.head()

,City_ID,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,Waitangi,NZ,1733443034,-43.9535,-176.5597,78,82,57.81,13.00
1,1,Bethel,US,1733443035,41.3712,-73.4140,49,63,35.01,5.01
2,2,Cabo San Lucas,MX,1733442969,22.8909,-109.9124,40,55,74.77,8.39
3,3,Olonkinbyen,SJ,1733443036,70.9221,-8.7187,100,99,28.74,21.92
4,4,Alotau,PG,1733443036,-10.3069,150.4462,100,67,86.95,3.58


In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,0,Waitangi,NZ,1733443034,-43.9535,-176.5597,78,82,57.81,13.00,
1,1,Bethel,US,1733443035,41.3712,-73.4140,49,63,35.01,5.01,
2,2,Cabo San Lucas,MX,1733442969,22.8909,-109.9124,40,55,74.77,8.39,
3,3,Olonkinbyen,SJ,1733443036,70.9221,-8.7187,100,99,28.74,21.92,
4,4,Alotau,PG,1733443036,-10.3069,150.4462,100,67,86.95,3.58,


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,0,Waitangi,NZ,1733443034,-43.9535,-176.5597,78,82,57.81,13.00,
1,1,Bethel,US,1733443035,41.3712,-73.4140,49,63,35.01,5.01,
2,2,Cabo San Lucas,MX,1733442969,22.8909,-109.9124,40,55,74.77,8.39,
3,3,Olonkinbyen,SJ,1733443036,70.9221,-8.7187,100,99,28.74,21.92,
4,4,Alotau,PG,1733443036,-10.3069,150.4462,100,67,86.95,3.58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
lat = (-90, 90)
lon = (-180, 180)
radius = 10000
limit = 20
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
categories = 'accommodation.hotel', 'accomodation', 'building.holiday_house'
params = {
    'categories':categories,
    'limit':limit,
    'filter':filters,
    'bias':bias,
    'apiKey':geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lat = row['Lat']
    lon=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_response = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError, TypeError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Waitangi - nearest hotel: No hotel found
Bethel - nearest hotel: No hotel found
Cabo San Lucas - nearest hotel: No hotel found
Olonkinbyen - nearest hotel: No hotel found
Alotau - nearest hotel: No hotel found
Uíge - nearest hotel: No hotel found
Enewetak - nearest hotel: No hotel found
Port Elizabeth - nearest hotel: No hotel found
Grytviken - nearest hotel: No hotel found
Cheongsong gun - nearest hotel: No hotel found
Miramichi - nearest hotel: No hotel found
Sitka - nearest hotel: No hotel found
Betio Village - nearest hotel: No hotel found
Invercargill - nearest hotel: No hotel found
Huazhou - nearest hotel: No hotel found
Whitehorse - nearest hotel: No hotel found
Tangalle - nearest hotel: No hotel found
Praia da Vitória - nearest hotel: No hotel found
Kerikeri - nearest hotel: No hotel found
Hawaiian Paradise Park - nearest hotel: No hotel found
Port-aux-Français - nearest hotel: No hotel found
Bikenibeu Village - nearest hotel: No hotel found
Lodja - neares

,City_ID,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,0,Waitangi,NZ,1733443034,-43.9535,-176.5597,78,82,57.81,13.00,No hotel found
1,1,Bethel,US,1733443035,41.3712,-73.4140,49,63,35.01,5.01,No hotel found
2,2,Cabo San Lucas,MX,1733442969,22.8909,-109.9124,40,55,74.77,8.39,No hotel found
3,3,Olonkinbyen,SJ,1733443036,70.9221,-8.7187,100,99,28.74,21.92,No hotel found
4,4,Alotau,PG,1733443036,-10.3069,150.4462,100,67,86.95,3.58,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
575,575,La Sierpe,CU,1733443117,21.7717,-79.2684,100,73,69.08,10.38,No hotel found
576,576,Sarupathar,IN,1733443117,26.1984,93.8094,39,86,59.27,0.40,No hotel found
577,577,Betioky,MG,1733443117,-23.7167,44.3833,13,70,74.12,3.56,No hotel found
580,580,Pariaman,ID,1733443118,-0.6190,100.1200,99,77,75.25,3.36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2 = citydata.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City",
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)